### Will a Customer Accept the Coupon?

**Context**

Imagine driving through town and a coupon is delivered to your cell phone for a restaraunt near where you are driving. Would you accept that coupon and take a short detour to the restaraunt? Would you accept the coupon but use it on a sunbsequent trip? Would you ignore the coupon entirely? What if the coupon was for a bar instead of a restaraunt? What about a coffee house? Would you accept a bar coupon with a minor passenger in the car? What about if it was just you and your partner in the car? Would weather impact the rate of acceptance? What about the time of day?

Obviously, proximity to the business is a factor on whether the coupon is delivered to the driver or not, but what are the factors that determine whether a driver accepts the coupon once it is delivered to them? How would you determine whether a driver is likely to accept a coupon?

**Overview**

The goal of this project is to use what you know about visualizations and probability distributions to distinguish between customers who accepted a driving coupon versus those that did not.

**Data**

This data comes to us from the UCI Machine Learning repository and was collected via a survey on Amazon Mechanical Turk. The survey describes different driving scenarios including the destination, current time, weather, passenger, etc., and then ask the person whether he will accept the coupon if he is the driver. Answers that the user will drive there ‘right away’ or ‘later before the coupon expires’ are labeled as ‘Y = 1’ and answers ‘no, I do not want the coupon’ are labeled as ‘Y = 0’.  There are five different types of coupons -- less expensive restaurants (under \\$20), coffee houses, carry out & take away, bar, and more expensive restaurants (\\$20 - \\$50). 

**Deliverables**

Your final product should be a brief report that highlights the differences between customers who did and did not accept the coupons.  To explore the data you will utilize your knowledge of plotting, statistical summaries, and visualization using Python. You will publish your findings in a public facing github repository as your first portfolio piece. 





### Data Description
Keep in mind that these values mentioned below are average values.

The attributes of this data set include:
1. User attributes
    -  Gender: male, female
    -  Age: below 21, 21 to 25, 26 to 30, etc.
    -  Marital Status: single, married partner, unmarried partner, or widowed
    -  Number of children: 0, 1, or more than 1
    -  Education: high school, bachelors degree, associates degree, or graduate degree
    -  Occupation: architecture & engineering, business & financial, etc.
    -  Annual income: less than \\$12500, \\$12500 - \\$24999, \\$25000 - \\$37499, etc.
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she buys takeaway food: 0, less than 1, 1 to 3, 4 to 8 or greater
    than 8
    -  Number of times that he/she goes to a coffee house: 0, less than 1, 1 to 3, 4 to 8 or
    greater than 8
    -  Number of times that he/she eats at a restaurant with average expense less than \\$20 per
    person: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    

2. Contextual attributes
    - Driving destination: home, work, or no urgent destination
    - Location of user, coupon and destination: we provide a map to show the geographical
    location of the user, destination, and the venue, and we mark the distance between each
    two places with time of driving. The user can see whether the venue is in the same
    direction as the destination.
    - Weather: sunny, rainy, or snowy
    - Temperature: 30F, 55F, or 80F
    - Time: 10AM, 2PM, or 6PM
    - Passenger: alone, partner, kid(s), or friend(s)


3. Coupon attributes
    - time before it expires: 2 hours or one day

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.express as px

### Problems

Use the prompts below to get started with your data analysis.  

1. Read in the `coupons.csv` file.




In [5]:
def trimurl(u):
  url1=u[:u.index('/view')]
  url2 = url1.replace(url1[url1.index('file/d/'):url1.index('file/d/')+7],'uc?export=download&id=')
  return url2
data = pd.read_csv(trimurl('https://drive.google.com/file/d/11p2XPYbpA_zLBzse6FoOi5sXcBox4NYb/view?usp=sharing'))

In [7]:
data.sample(10)

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
11506,Home,Alone,Sunny,80,6PM,Carry out & Take away,2h,Female,below21,Single,...,1~3,less1,4~8,gt8,1,1,0,1,0,1
5802,Work,Alone,Sunny,80,7AM,Coffee House,2h,Female,31,Unmarried partner,...,less1,1~3,4~8,1~3,1,0,0,1,0,0
3674,No Urgent Place,Friend(s),Sunny,55,2PM,Carry out & Take away,1d,Male,26,Married partner,...,1~3,1~3,1~3,less1,1,1,0,0,1,1
9567,Home,Alone,Sunny,80,6PM,Restaurant(<20),1d,Male,31,Married partner,...,less1,less1,1~3,less1,1,0,0,1,0,1
12595,No Urgent Place,Alone,Rainy,55,10AM,Bar,1d,Female,50plus,Divorced,...,less1,1~3,1~3,less1,1,1,0,0,1,0
7786,Home,Alone,Rainy,55,10PM,Coffee House,1d,Male,26,Single,...,gt8,gt8,gt8,gt8,1,0,0,1,0,1
5342,Work,Alone,Sunny,80,7AM,Coffee House,2h,Female,26,Unmarried partner,...,less1,1~3,1~3,less1,1,0,0,1,0,0
3614,No Urgent Place,Alone,Sunny,55,10AM,Coffee House,2h,Female,26,Single,...,less1,less1,1~3,less1,1,0,0,0,1,0
2850,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Single,...,less1,1~3,less1,less1,1,1,0,0,1,0
11782,No Urgent Place,Friend(s),Sunny,80,10PM,Bar,1d,Female,41,Married partner,...,less1,less1,less1,less1,1,1,0,0,1,0


2. Investigate the dataset for missing or problematic data.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12684 entries, 0 to 12683
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12684 non-null  object
 1   passanger             12684 non-null  object
 2   weather               12684 non-null  object
 3   temperature           12684 non-null  int64 
 4   time                  12684 non-null  object
 5   coupon                12684 non-null  object
 6   expiration            12684 non-null  object
 7   gender                12684 non-null  object
 8   age                   12684 non-null  object
 9   maritalStatus         12684 non-null  object
 10  has_children          12684 non-null  int64 
 11  education             12684 non-null  object
 12  occupation            12684 non-null  object
 13  income                12684 non-null  object
 14  car                   108 non-null    object
 15  Bar                   12577 non-null

**Comment:** Many variables with ```NA``` or ```NaN``` values. Need to view the unique outputs of each variable and group into appropriate outputs and datatypes for analysis

3. Decide what to do about your missing data -- drop, replace, other...

In [13]:
data['destination'].value_counts()

No Urgent Place    6283
Home               3237
Work               3164
Name: destination, dtype: int64

In [14]:
data=data.rename(columns={'passanger':'passenger'})
data['passenger'].value_counts()

Alone        7305
Friend(s)    3298
Partner      1075
Kid(s)       1006
Name: passenger, dtype: int64

In [15]:
data['weather'].value_counts()

Sunny    10069
Snowy     1405
Rainy     1210
Name: weather, dtype: int64

In [17]:
data['temperature'].value_counts()

80    6528
55    3840
30    2316
Name: temperature, dtype: int64

In [18]:
data['time'].value_counts()

6PM     3230
7AM     3164
10AM    2275
2PM     2009
10PM    2006
Name: time, dtype: int64

In [19]:
data['coupon'].value_counts()

Coffee House             3996
Restaurant(<20)          2786
Carry out & Take away    2393
Bar                      2017
Restaurant(20-50)        1492
Name: coupon, dtype: int64

In [20]:
data['expiration'].value_counts()

1d    7091
2h    5593
Name: expiration, dtype: int64

In [21]:
data['gender'].value_counts()

Female    6511
Male      6173
Name: gender, dtype: int64

In [29]:
data['age'].value_counts()
data['age']=data['age'].replace('50plus','50').replace('below21','16')
data['age']=data['age'].astype(int)

numpy.int32

In [30]:
data['maritalStatus'].value_counts()

Married partner      5100
Single               4752
Unmarried partner    2186
Divorced              516
Widowed               130
Name: maritalStatus, dtype: int64

In [32]:
data['has_children'].value_counts()

0    7431
1    5253
Name: has_children, dtype: int64

4. What proportion of the total observations chose to accept the coupon? 



In [36]:
data['education'].value_counts()

Some college - no degree                  4351
Bachelors degree                          4335
Graduate degree (Masters or Doctorate)    1852
Associates degree                         1153
High School Graduate                       905
Some High School                            88
Name: education, dtype: int64

In [33]:
data['occupation'].value_counts()

Unemployed                                   1870
Student                                      1584
Computer & Mathematical                      1408
Sales & Related                              1093
Education&Training&Library                    943
Management                                    838
Office & Administrative Support               639
Arts Design Entertainment Sports & Media      629
Business & Financial                          544
Retired                                       495
Food Preparation & Serving Related            298
Healthcare Practitioners & Technical          244
Healthcare Support                            242
Community & Social Services                   241
Legal                                         219
Transportation & Material Moving              218
Architecture & Engineering                    175
Personal Care & Service                       175
Protective Service                            175
Life Physical Social Science                  170


In [35]:
data['income'].value_counts()
type(data['income'][0])

str

In [37]:
data['car'].value_counts()

Scooter and motorcycle                      22
Mazda5                                      22
do not drive                                22
crossover                                   21
Car that is too old to install Onstar :D    21
Name: car, dtype: int64

**Assumption:** The prompt states that these are *driving* coupons, i.e. all participants in the survey are drivers of a vehicle. Therefore assume ```NA``` / ```NaN``` values mean 'car' is True

**Assumption:** Less than 1% (108 / 12,684) of participants specified the type of car they drove. Therefore, this column of data has a low probability of impacting analysis outcome. Will remove from dataset.

In [39]:
data=data.drop('car',axis=1)

In [43]:
data['Bar'].value_counts()
#len(data['Bar'])

never    5197
less1    3482
1~3      2473
4~8      1076
gt8       349
Name: Bar, dtype: int64

**Assumption:** ```NA``` or ```NaN``` values for ```Bar``` column mean ```never```

In [46]:
data['Bar']=data['Bar'].fillna('never')
data['CoffeHouse']=data['CoffeHouse'].fillna('never')


5. Use a bar plot to visualize the `coupon` column.

6. Use a histogram to visualize the temperature column.

**Investigating the Bar Coupons**

Now, we will lead you through an exploration of just the bar related coupons.  

1. Create a new `DataFrame` that contains just the bar coupons.


2. What proportion of bar coupons were accepted?


3. Compare the acceptance rate between those who went to a bar 3 or fewer times a month to those who went more.


4. Compare the acceptance rate between drivers who go to a bar more than once a month and are over the age of 25 to the all others.  Is there a difference?


5. Use the same process to compare the acceptance rate between drivers who go to bars more than once a month and had passengers that were not a kid and had occupations other than farming, fishing, or forestry. 


6. Compare the acceptance rates between those drivers who:

- go to bars more than once a month, had passengers that were not a kid, and were not widowed *OR*
- go to bars more than once a month and are under the age of 30 *OR*
- go to cheap restaurants more than 4 times a month and income is less than 50K. 



7.  Based on these observations, what do you hypothesize about drivers who accepted the bar coupons?

### Independent Investigation

Using the bar coupon example as motivation, you are to explore one of the other coupon groups and try to determine the characteristics of passengers who accept the coupons.  